<a href="https://colab.research.google.com/github/linamarwabelkilani/modelisation-hydrologique/blob/main/model_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importation des données depuis github et preparation des types pour le model

In [1]:
#import dataset from github
import pandas as pd
data_url = "https://raw.githubusercontent.com/linamarwabelkilani/modelisation-hydrologique/main/Dataset.csv"
df = pd.read_csv(data_url, on_bad_lines='skip',sep=";")


df['Station']=df['Station'].astype(str)
df = df.astype({'Station':'string'})
df = df.astype({'Date':'string'})
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True,errors='coerce')
df = df.dropna()
print(df.dtypes)
df

Date             datetime64[ns]
Station                  string
Precipitation           float64
dtype: object


,Date,Station,Precipitation
0,1951-06-01,AIN BEYA OUED RHEZAL,0.0
1,1951-06-02,AIN BEYA OUED RHEZAL,0.0
2,1951-06-03,AIN BEYA OUED RHEZAL,0.0
3,1951-06-04,AIN BEYA OUED RHEZAL,0.0
4,1951-06-05,AIN BEYA OUED RHEZAL,0.0
...,...,...,...
2103688,2015-08-27,ZOUARINE GARE,3.0
2103689,2015-08-28,ZOUARINE GARE,7.0
2103690,2015-08-29,ZOUARINE GARE,0.0
2103691,2015-08-30,ZOUARINE GARE,0.0


preparation de model 

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

#creating model
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])

x=df["Date"][:100000]
y=df["Precipitation"][:100000]

x_train = np.asarray(x).astype(np.float)
y_train = np.asarray(y).astype(np.float)

model.compile(optimizer='sgd', loss='mean_squared_error')

#train the model
model.fit(x_train, y_train, epochs=10)




Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if sys.path[0] == '':


3125/3125 [==============================] - 7s 2ms/step - loss: nan
Epoch 2/10
3125/3125 [==============================] - 4s 1ms/step - loss: nan
Epoch 3/10
3125/3125 [==============================] - 4s 1ms/step - loss: nan
Epoch 4/10
3125/3125 [==============================] - 4s 1ms/step - loss: nan
Epoch 5/10
3125/3125 [==============================] - 4s 1ms/step - loss: nan
Epoch 6/10
3125/3125 [==============================] - 4s 1ms/step - loss: nan
Epoch 7/10
3125/3125 [==============================] - 4s 1ms/step - loss: nan
Epoch 8/10
3125/3125 [==============================] - 4s 1ms/step - loss: nan
Epoch 9/10
3125/3125 [==============================] - 4s 1ms/step - loss: nan
Epoch 10/10
3125/3125 [==============================] - 4s 1ms/step - loss: nan


In [4]:
#Using Model
x_test = np.asarray(df["Date"][100000:1000010]).astype(np.float32)
print(model.predict(x_test))

[[nan]
 [nan]
 [nan]
 ...
 [nan]
 [nan]
 [nan]]
